# 제 1고지
## step 01

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data

In [2]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(data)

1.0


In [4]:
x.data = np.array(2.0)
print(x.data)

2.0


In [5]:
a = np.array(1)
a.ndim

0

In [6]:
b = np.array([1, 2, 3])
b.ndim

1

In [8]:
c = np.array([[1, 2, 3],
            [4, 5, 6]])
c.ndim

2

## step 02

In [9]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2
        output = Variable(y)
        return output

In [10]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [11]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    def forward(self, x):
        raise NotImplementedError()

In [12]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [13]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## step 03

In [14]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [15]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## step 04

In [16]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [17]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [18]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


## step 05(이론)
## step 06

In [20]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

In [21]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # memorize input 
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):
        raise NotImplementedError()

In [23]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [24]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [28]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

print("input = ", x.data, "\na = ", a.data, "\nb = ", b.data, "\noutput = ", y.data)

input =  0.5 
a =  0.25 
b =  1.2840254166877414 
output =  1.648721270700128


In [29]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


## step 07

In [55]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func

In [56]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # make relationship between variable and fuction
        self.input = input
        self.output = output # memorize output
        return output

In [57]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [58]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [59]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print("input = ", x.data, "\na = ", a.data, "\nb = ", b.data, "\noutput = ", y.data)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

input =  0.5 
a =  0.25 
b =  1.2840254166877414 
output =  1.648721270700128


In [60]:
y.grad = np.array(1.0)

C = y.creator # get function
b = C.input # get input
b.grad = C.backward(y.grad) # call backward

In [61]:
B = b.creator # get function
a = B.input # get input
a.grad = B.backward(b.grad) # call backward

In [62]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


In [63]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator # get function
        if f is not None:
            x = f.input # get input
            x.grad = f.backward(self.grad) # call backward
            x.backward() # recurrent

In [65]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print("input = ", x.data, "\na = ", a.data, "\nb = ", b.data, "\noutput = ", y.data)

y.grad = np.array(1.0)
y.backward()
print("dy/dx = ", x.grad)

input =  0.5 
a =  0.25 
b =  1.2840254166877414 
output =  1.648721270700128
dy/dx =  3.297442541400256


## step 08

In [66]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        # iteration
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # get function
            x, y = f.input, f.output # get input and output
            x.grad = f.backward(y.grad) # call backward
            
            if x.creator is not None:
                funcs.append(x.creator)

In [67]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print("input = ", x.data, "\na = ", a.data, "\nb = ", b.data, "\noutput = ", y.data)

y.grad = np.array(1.0)
y.backward()
print("dy/dx = ", x.grad)

input =  0.5 
a =  0.25 
b =  1.2840254166877414 
output =  1.648721270700128
dy/dx =  3.297442541400256


## step 09

In [68]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [69]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print("dy/dx = ", x.grad)

dy/dx =  3.297442541400256


In [70]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print("dy/dx = ", x.grad)

dy/dx =  3.297442541400256


In [71]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        #--------------------------------------
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        #--------------------------------------
        
        # iteration
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # get function
            x, y = f.input, f.output # get input and output
            x.grad = f.backward(y.grad) # call backward
            
            if x.creator is not None:
                funcs.append(x.creator)

In [72]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print("dy/dx = ", x.grad)

dy/dx =  3.297442541400256


In [93]:
class Variable:
    def __init__(self, data):
        #------------------------------------------------------------------
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not acceptable.".format(type(data)))
        #------------------------------------------------------------------        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        # iteration
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # get function
            x, y = f.input, f.output # get input and output
            x.grad = f.backward(y.grad) # call backward
            
            if x.creator is not None:
                funcs.append(x.creator)

In [74]:
tmp = Variable(np.array(1.3))

In [75]:
tmp = Variable(None)

In [76]:
tmp = Variable(1.2)

TypeError: <class 'float'> is not acceptable.

In [77]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [78]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [79]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [80]:
np.isscalar(np.float64(1.0))

True

In [81]:
np.isscalar(np.array(1.6))

False

In [82]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y)) # add as_array function!
        output.set_creator(self) 
        self.input = input
        self.output = output 
        return output

## step 10

In [98]:
%%writefile forward_backward_test.py
import unittest
import numpy as np

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self) 
        self.input = input
        self.output = output 
        return output

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
def square(x):
    return Square()(x)
    
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not acceptable.".format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output 
            x.grad = f.backward(y.grad) 
            
            if x.creator is not None:
                funcs.append(x.creator)

                
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
        
unittest.main()

Writing forward_backward_test.py


In [99]:
%run forward_backward_test.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.013s

OK


In [102]:
%%writefile gradient_checking.py
import unittest
import numpy as np

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self) 
        self.input = input
        self.output = output 
        return output

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
def square(x):
    return Square()(x)
    
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not acceptable.".format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output 
            x.grad = f.backward(y.grad) 
            
            if x.creator is not None:
                funcs.append(x.creator)

                
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
        
    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
        
unittest.main()

Writing gradient_checking.py


In [103]:
%run gradient_checking.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.022s

OK
